In [1]:
import os
from time import time
import tensorflow as tf

def data_generator(features,labels,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((tf.cast((features/255),tf.float32),labels))
  dataset = dataset.shuffle(buffer_size=len(labels)+1)
  dataset = dataset.batch(batch_size=batch_size,
                          drop_remainder=True)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

def model():  
  model = tf.keras.applications.Xception(include_top=False,
                                            weights='imagenet')
  x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
  predictions = tf.keras.layers.Dense(num_classes,activation="softmax")(x)
  return tf.keras.Model(inputs=model.input, outputs=predictions)

if __name__ == "__main__":
  batch_size = 32
  epochs = 2
  num_classes = 10

  (x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

  train_dataset = data_generator(x_train, y_train, batch_size)
  test_dataset = data_generator(x_test, y_test, batch_size)

  xception = model()
  xception.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=["accuracy"])

  start_time = time()
  xception.fit(train_dataset, epochs=epochs, steps_per_epoch=len(x_train)//batch_size,
              validation_data=test_dataset,validation_steps=len(x_test)//batch_size)
  end_time = time()

  print("Time without xla and mpt",end_time-start_time)

Epoch 1/2
1562/1562 [==============================] - 101s 61ms/step - loss: 1.8690 - accuracy: 0.2933 - val_loss: 0.8928 - val_accuracy: 0.6964
Epoch 2/2
1562/1562 [==============================] - 94s 60ms/step - loss: 0.8278 - accuracy: 0.7258 - val_loss: 0.7259 - val_accuracy: 0.7563
Time without xla and mpt 195.7514898777008


In [2]:
import os
from time import time
import tensorflow as tf

def data_generator(features,labels,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((tf.cast((features/255),tf.float16),tf.cast(labels,tf.float16)))
  dataset = dataset.shuffle(buffer_size=len(labels)+1)
  dataset = dataset.batch(batch_size=batch_size,
                          drop_remainder=True)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

def model():  
  model = tf.keras.applications.Xception(include_top=False,
                                            weights='imagenet')
  x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
  x = tf.keras.layers.Dense(num_classes)(x)
  predictions = tf.keras.layers.Activation('softmax', dtype=tf.float32)(x)
  return tf.keras.Model(inputs=model.input, outputs=predictions)

if __name__ == "__main__":

  os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit"
  tf.config.optimizer.set_jit(True)

  policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
  tf.keras.mixed_precision.experimental.set_policy(policy)

  batch_size = 32
  epochs = 2
  num_classes = 10

  (x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

  train_dataset = data_generator(x_train, y_train, batch_size)
  test_dataset = data_generator(x_test, y_test, batch_size)

  optimizer = tf.optimizers.Adam()
  optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(optimizer, loss_scale='dynamic')

  xception = model()
  xception.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=["accuracy"])

  start_time = time()
  xception.fit(train_dataset, epochs=epochs, steps_per_epoch=len(x_train)//batch_size,
              validation_data=test_dataset,validation_steps=len(x_test)//batch_size)
  end_time = time()
  print("Time with xla and mpt",end_time-start_time)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)
Epoch 1/2
1562/1562 [==============================] - 67s 28ms/step - loss: 2.3043 - accuracy: 0.1024 - val_loss: 2.3062 - val_accuracy: 0.0955
Epoch 2/2
1562/1562 [==============================] - 40s 26ms/step - loss: 2.3051 - accuracy: 0.1021 - val_loss: 2.3041 - val_accuracy: 0.0978
Time with xla and mpt 107.1470365524292
